In [2]:
import numpy as np

import xarray
import dask

In [8]:
ssp585 = xarray.open_zarr("https://api.cloud.ifca.es:8080/swift/v1/IPCC-IA-Monthly/zarr/CMIP6/historical/t_CMIP6_historical_mon_185001-201412")
ssp585["t"]

<xarray.DataArray 't' (member: 35, time: 1980, lat: 180, lon: 360)> Size: 18GB
dask.array<open_dataset-t, shape=(35, 1980, 180, 360), dtype=float32, chunksize=(35, 1, 180, 360), chunktype=numpy.ndarray>
Coordinates:
    height2m  float64 8B ...
  * lat       (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon       (lon) float64 3kB -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * member    (member) <U45 6kB 'CSIRO-ARCCSS_ACCESS-CM2_r1i1p1f1' ... 'MOHC_...
  * time      (time) datetime64[ns] 16kB 1850-01-01 1850-02-01 ... 2014-12-01
Attributes:
    cell_methods:   time: mean within days time: mean over days area: mean
    comment:        Monthly mean of daily mean near-surface (usually, 2 meter...
    grid_mapping:   crs
    long_name:      Monthly mean of daily mean temperature
    standard_name:  air_temperature
    units:          degC

In [9]:
weights = np.cos(np.deg2rad(ssp585["lat"]))

In [13]:
%%time

a = ssp585["t"].chunk({"member": -1, "time": 1}).weighted(weights).mean(["member", "lat", "lon"]).compute(num_workers=8, scheduler="processes")

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x74700d85d9d0>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x74700d6268d0>, 2439219.530693094)]']
connector: <aiohttp.connector.TCPConnector object at 0x74700d89f7a0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x77e254bf7e60>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x77e254c528d0>, 2439218.786898845)]']
connector: <aiohttp.connector.TCPConnector object at 0x77e254c6e0f0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x788e65b5df10>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x788e5fa0e8d0>, 2439218.771307963)]']
connector: <aiohttp.connector.TCPConnector object at 0x788e5fa262a0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x771288fde180>
Unclosed connector
connections: ['[(<aio

CPU times: user 1min 12s, sys: 1min 3s, total: 2min 16s
Wall time: 3min 20s


In [12]:
%%time

b = ssp585["t"].chunk({"member": -1, "time": 1}).mean(["member", "lat", "lon"]).compute(num_workers=8, scheduler="processes")

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x71c97483be60>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x71c96f646930>, 2438678.689051773)]']
connector: <aiohttp.connector.TCPConnector object at 0x71c96f66a780>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x746d346b2060>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x746d2fb2e930>, 2438679.545292864)]']
connector: <aiohttp.connector.TCPConnector object at 0x746d2fb56810>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x70d4aa0799d0>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x70d4a8e7e930>, 2438679.024356359)]']
connector: <aiohttp.connector.TCPConnector object at 0x70d4a8ea2b10>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7268ed4db1a0>
Unclosed connector
connections: ['[(<aio

CPU times: user 2.87 s, sys: 173 ms, total: 3.05 s
Wall time: 1min 24s


In [15]:
%%time

a = ssp585["t"].chunk({"member": -1, "time": 1}).compute(num_workers=8, scheduler="processes")

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7d555de65ee0>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7d555de4e8d0>, 2441332.943202671)]']
connector: <aiohttp.connector.TCPConnector object at 0x7d555de661b0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f4882786030>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7f48825268d0>, 2441333.19844196)]']
connector: <aiohttp.connector.TCPConnector object at 0x7f4928354e60>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x77a96895a2a0>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x77a968bd68d0>, 2441333.585625991)]']
connector: <aiohttp.connector.TCPConnector object at 0x77a96903e420>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x733f712dd4c0>
Unclosed connector
connections: ['[(<aioh

CPU times: user 27 s, sys: 50.1 s, total: 1min 17s
Wall time: 1min 55s


In [16]:
%%time

b = a.weighted(weights).mean(["member", "lat", "lon"])

CPU times: user 30.5 s, sys: 11.9 s, total: 42.4 s
Wall time: 42.7 s
